In [26]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
 
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        #consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
        #consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        #access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        #access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
        
         
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) )
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
            #print (tweet.created_at)
            #print(tweet.text)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
                #print (tweet.created_at)
                #print(tweet.text)
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                parsed_tweet['created_at'] = tweet.created_at
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
 
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    query = "Bitcoin since:2018-03-01 until:2018-03-03"
    tweets = api.get_tweets(query , count = 200)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
   
    # printing first 5 positive tweets
    print("Positive tweets:")
    for tweet in ptweets[:10]:
        print("Created at: ", tweet['created_at'].strftime("%Y-%m-%d" ))
        print(tweet['text'])
 
    # printing first 5 negative tweets
   # print("Negative tweets:")
   # for tweet in ntweets[:10]:
    #    print(tweet['text'])
 
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 42.391304347826086 %
Negative tweets percentage: 10.869565217391305 %
Positive tweets:
Created at:  2018-03-02
RT @21Cryptos: Shill us your top coins!

What's your no.1 holding?

https://t.co/QkskTI9kMy 
#bitcoin #crypto #altcoin https://t.co/NZcBafB…
Created at:  2018-03-02
RT @bethereumteam: We've just hit 500 members on Telegram. Have you seen our Group Butler? He's quite funny.
#Bethereum #crypto #bitcoin #t…
Created at:  2018-03-02
Learn how to earn free #bitcion &amp; #ETH via #airdrop. Join the channel 
https://t.co/wtf7st95uK …
#crypto #altcoin… https://t.co/tEGtCCRA6I
Created at:  2018-03-02
The Lightning network may be the most under appreciated tech since Bitcoin itself.
Created at:  2018-03-02
Come to join a good project
Pumpkin Pay —is now giving free airdrop!!
Follow @PumpkinPayTeam  get 100 PPKC first an… https://t.co/oOi2AKx9Lm
Created at:  2018-03-02
RT @OfficialXAOS: XAOS PRE-SALE LIVE! ONLY $0.10! JOIN THE REVOLUTION
https://t.co/qLAe3Gd1f